In [282]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

In [283]:
artista = 'PIBCuritiba'

In [284]:
colunas_data = {
    'Impressões':'Impressões',
    'Visualizações':'Visualizações',
    'Taxa de cliques de impressões (%)':'CTR (%)',
    'Duração média da visualização':'Tempo Médio Assistido',
    'Porcentagem visualizada média (%)':'Porcentagem Média Assistida',
    'RPM (USD)':'RPM (USD)',
    'Marcações "Não gostei"':'Porcentagem de "Não Gostei"',
    'Inscritos':'Saldo de Inscritos'
}

colunas_conteudo = {
    'Vídeos publicados':'Número de conteúdos Novo',
    'Impressões':'Impressões Novo',
    'Visualizações':'Visualizações Novo',
    'Taxa de cliques de impressões (%)':'CTR (%) Novo',
    'Duração média da visualização':'Tempo Médio Assistido Novo',
    'Porcentagem visualizada média (%)':'Porcentagem Média Assistida Novo',
    'RPM (USD)':'RPM (USD) Novo',
    'Marcações "Não gostei"':'Porcentagem de "Não Gostei" Novo',
    'Inscritos':'Saldo de Inscritos Novo',
    'Minutagem média':'Tamanho médio do vídeo (min) Novo'
    
}

colunas_receita = {
    'Receita estimada (USD)_x':'Receita Total',
    'Receita estimada (USD)_y':'Receita Conteúdo novo (Soma)',
    'Conteúdo Velho':'Receita Conteúdo velho (Soma)'

}

colunas_shorts = {
    'Vídeos publicados':'Número de Shorts',
    'Impressões':'Impressões Shorts',
    'Visualizações':'Visualizações Shorts',
    'Porcentagem visualizada média (%)':'Retenção (%) Shorts',
    'RPM (USD)':'RPM Shorts',
    'Duração média da visualização':'Tempo Médio Assistido Shorts',
    'Minutagem média':'Tamanho de Vídeo Shorts',
    'Inscritos':'Saldo de Inscritos Shorts'
}

colunas_comunidade = {
    'Post text':'Texto da postagem',
    'Texto da publicação':'Texto da postagem',
    'Post publish time':'Horário de publicação da postagem',
    'Hora da publicação':'Horário de publicação da postagem',
    'Post impressions':'Impressões da postagem',
    'Impressões da publicação':'Impressões da postagem',
    'Post likes':'"Gostei" da postagem',
    'Gostos da publicação':'"Gostei" da postagem',
    'Post like rate (%)':'Média de "Gostei" da postagem (%)',
    """Média de """"Gostei"""" da postagem (%)""":'Média de "Gostei" da postagem (%)',
    'Percentagem de Gostos da publicação (%)':'Média de "Gostei" da postagem (%)',

}

colunas_receita_ordenada = ['Receita Conteúdo novo (Soma)', 'Receita Conteúdo velho (Soma)', 'Receita Total'] 

In [285]:
def format_as_percentage(value):
    return '{:.2%}'.format(value)

In [286]:
f = open("dados_full/"+artista+"/sub.txt", "r")
subs = int(''.join((f.readline()).split('.')))
f.close()

# Tabela de Análise de Conteúdo (DATA/SUBS)

In [287]:
data_1 = pd.read_csv(f'dados_full/{artista}/lives.csv')
data_1 = data_1.drop(data_1.index[0])
data_1['Data'] = pd.to_datetime(data_1['Data'])
data_1['Data'] = data_1['Data'].dt.to_period('M')
data = data_1.copy()
data = data[['Data','Impressões', 'Visualizações','Taxa de cliques de impressões (%)', 'Duração média da visualização', 'Porcentagem visualizada média (%)', 'RPM (USD)', 'Marcações "Não gostei"', 'Inscritos']]
data['Marcações "Não gostei"'] = (data['Marcações "Não gostei"'] / data['Visualizações'])
data['Duração média da visualização'] = pd.to_timedelta(data['Duração média da visualização'])
data['Taxa de cliques de impressões (%)'] = data['Taxa de cliques de impressões (%)'] / 100
data['Porcentagem visualizada média (%)'] = data['Porcentagem visualizada média (%)'] / 100
data_mes = data.copy()
data_mes.sort_values(by='Data',inplace=True, ascending= False)
mes = data_mes['Data'].iloc[0]


data_impressoes = data['Impressões'].mean()
data_visualizacoes = data['Visualizações'].mean()
data_ctr = data['Taxa de cliques de impressões (%)'].mean()
data_tempo_assistido = data['Duração média da visualização'].mean()
data_tempo_assistido = data_tempo_assistido.seconds
data_tempo_assistido = '{:02d}:{:02d}'.format(data_tempo_assistido // 60, data_tempo_assistido % 60)
data_porcentagem_assistida = data['Porcentagem visualizada média (%)'].mean()
data_rpm = data['RPM (USD)'].mean()
data_nao_gostei = data['Marcações "Não gostei"'].mean()


data['Duração média da visualização'] = pd.to_timedelta(data['Duração média da visualização']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(data['Duração média da visualização']).dt.components['seconds'].astype(str).str.zfill(2)
data.sort_values(by='Data', inplace=True, ascending=False)
data_cumulative = subs - data['Inscritos'].cumsum()
data['Inscritos'] = data_cumulative.shift(1).fillna(subs).astype(int)
data_inscritos = data['Inscritos'].mean()
data.sort_values(by='Data', inplace=True, ascending=True)
data.rename(columns=colunas_data, inplace=True)
data.set_index('Data', inplace = True)
data = data.T
data.insert(0, 'Média', [data_impressoes, data_visualizacoes, data_ctr, data_tempo_assistido, data_porcentagem_assistida, data_rpm, data_nao_gostei, data_inscritos])
data.reset_index(inplace= True)
data.rename(columns={'index':'Análise Conteúdos'}, inplace = True)
data.fillna(0, inplace=True)
data

Data,Análise Conteúdos,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Impressões,1688792.5,1890242,1650970,1571296,1847433,1616212,1556602
1,Visualizações,169166.333333,180182,169352,150475,180640,167898,166451
2,CTR (%),0.0686,0.0642,0.0688,0.0663,0.0677,0.0719,0.0727
3,Tempo Médio Assistido,22:13,21:47,20:41,22:29,22:31,22:51,22:59
4,Porcentagem Média Assistida,0.1974,0.198,0.1871,0.2029,0.1898,0.2027,0.2039
5,RPM (USD),1.091167,0.822,1.44,1.051,0.952,1.148,1.134
6,"Porcentagem de ""Não Gostei""",0.000341,0.000277,0.000313,0.000272,0.000332,0.000405,0.000445
7,Saldo de Inscritos,146833.5,145630,146067,146539,147052,147572,148141


In [288]:
data_temp = data.copy()
data_temp.set_index('Análise Conteúdos', inplace = True)
data_temp.loc['Tempo Médio Assistido'] = data_temp.loc['Tempo Médio Assistido'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))

data_desvio = data_temp.copy()
data_desvio.iloc[:, 1:] = (data_desvio.iloc[:, 1:].div(data_desvio['Média'], axis=0) - 1)
data_desvio.drop('Média', axis=1, inplace=True)
data_desvio.reset_index(drop=True, inplace=True)
data_desvio.insert(0, 'Análise Conteúdos', data['Análise Conteúdos'])
data_desvio.fillna(0, inplace=True)
data_desvio.iloc[:, 1:] = data_desvio.iloc[:, 1:].replace('.', ',', regex=True)
data_desvio.iloc[:, 1:] = data_desvio.iloc[:, 1:].astype(float)

In [289]:
#Formatação de porcentagens conteúdo
data.set_index('Análise Conteúdos', inplace=True)
data.loc['CTR (%)'] = data.loc['CTR (%)'].map(format_as_percentage)
data.loc['Porcentagem de "Não Gostei"'] = data.loc['Porcentagem de "Não Gostei"'].map(format_as_percentage)
data.loc['Porcentagem Média Assistida'] = data.loc['Porcentagem Média Assistida'].map(format_as_percentage)
data.reset_index(inplace=True)
data = data.astype(str)

#Formatação de porcentagens desvio
data_desvio.iloc[:, 1:] = data_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
data_desvio_formatado = data_desvio.iloc[:, 1:].map(format_as_percentage)
data_desvio_formatado = pd.concat([data_desvio['Análise Conteúdos'], data_desvio_formatado], axis=1)
data_desvio_formatado = data_desvio_formatado.astype(str)

In [290]:
data_desvio_formatado

,Análise Conteúdos,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Impressões,11.93%,-2.24%,-6.96%,9.39%,-4.30%,-7.83%
1,Visualizações,6.51%,0.11%,-11.05%,6.78%,-0.75%,-1.61%
2,CTR (%),-6.41%,0.29%,-3.35%,-1.31%,4.81%,5.98%
3,Tempo Médio Assistido,-1.95%,-6.90%,1.20%,1.35%,2.85%,3.45%
4,Porcentagem Média Assistida,0.30%,-5.22%,2.79%,-3.85%,2.68%,3.29%
5,RPM (USD),-24.67%,31.97%,-3.68%,-12.75%,5.21%,3.93%
6,"Porcentagem de ""Não Gostei""",-18.57%,-8.16%,-20.04%,-2.53%,18.85%,30.46%
7,Saldo de Inscritos,-0.82%,-0.52%,-0.20%,0.15%,0.50%,0.89%


# Tabela de Novos Conteúdos

In [291]:
def processar_dados(artista, num_videos):
    dados = []
    for i in range(1, num_videos + 1):
        arquivo = f'dados_full/{artista}/videos{i}.csv'
        df = pd.read_csv(arquivo)
        df['Horário de publicação do vídeo'] = pd.to_datetime(df['Horário de publicação do vídeo'])
        df['Data'] = df['Horário de publicação do vídeo'].dt.to_period('M')
        df.sort_values(by='Data', ascending=False, inplace=True)

        ultimo_mes = df['Horário de publicação do vídeo'].max().to_period('M')
        meses_selecionados = pd.period_range(end=ultimo_mes, periods=4, freq='M')
        df = df[df['Horário de publicação do vídeo'].dt.to_period('M').isin(meses_selecionados)]

        df['Duração média da visualização'] = pd.to_timedelta(df['Duração média da visualização']).apply(lambda x: x.total_seconds())
        df.set_index('Data', inplace=True)
        df = df.dropna(subset=['Horário de publicação do vídeo'])
        dados.append(df)
    return pd.concat(dados)

num_videos = 6
conteudo_1 = processar_dados(artista, num_videos)
conteudo_1.reset_index(inplace=True)
conteudo_1['Data'].unique()

<PeriodArray>
['2023-12', '2023-11', '2024-01', '2024-02', '2024-03', '2024-04']
Length: 6, dtype: period[M]

In [292]:
# Contagem de linhas por valor na coluna 'Data'
videos_publicados = conteudo_1['Data'].value_counts().reset_index()
videos_publicados.columns = ['Data', 'Vídeos Publicados']
# videos_publicados.loc[[1,2,3,4,5], 'Vídeos Publicados'] = 0
videos_publicados.sort_values(by='Data', inplace=True)
videos_publicados

,Data,Vídeos Publicados
0,2023-11,105
1,2023-12,82
5,2024-01,68
3,2024-02,73
4,2024-03,71
2,2024-04,80


In [293]:
conteudo_1.set_index('Data', inplace=True)
conteudo = conteudo_1.copy()
conteudo_1 = conteudo.groupby(pd.Grouper(freq='M')).agg({'Receita estimada (USD)' : 'sum'})
conteudo_1 = conteudo_1.reset_index()


conteudo = conteudo.groupby(pd.Grouper(freq='M')).agg({'Impressões':'sum', 'Visualizações':'sum', 'Taxa de cliques de impressões (%)':'mean', 'Duração média da visualização':'mean', 'Porcentagem visualizada média (%)':'mean', 'RPM (USD)':'mean', 'Marcações "Não gostei"':'mean', 'Inscritos':'sum'})
conteudo['Minutagem média'] = (conteudo['Duração média da visualização'] * 100) / conteudo['Porcentagem visualizada média (%)']
conteudo['Marcações "Não gostei"'] = (conteudo['Marcações "Não gostei"'] / conteudo['Visualizações']) * 100
conteudo['Taxa de cliques de impressões (%)'] = conteudo['Taxa de cliques de impressões (%)'] / 100
conteudo['Porcentagem visualizada média (%)'] = conteudo['Porcentagem visualizada média (%)'] / 100
conteudo_2 = conteudo.copy()

conteudo_impressoes = conteudo['Impressões'].mean()
conteudo_visualizacoes = conteudo['Visualizações'].mean()
conteudo_ctr = conteudo['Taxa de cliques de impressões (%)'].mean()
conteudo_tempo_assistido = conteudo['Duração média da visualização'].mean()
conteudo_tempo_assistido = '{:02d}:{:02d}'.format(int(conteudo_tempo_assistido // 60), int(conteudo_tempo_assistido % 60))
conteudo_porcentagem_assistida = conteudo['Porcentagem visualizada média (%)'].mean()
conteudo_rpm = conteudo['RPM (USD)'].mean()
conteudo_nao_gostei = conteudo['Marcações "Não gostei"'].mean()
conteudo_inscritos = conteudo['Inscritos'].mean()


conteudo['Duração média da visualização'].fillna(0, inplace=True)
conteudo['Duração média da visualização'] = conteudo['Duração média da visualização'].astype(int)
conteudo['Duração média da visualização'] = conteudo['Duração média da visualização'].apply(lambda x: int(x))
conteudo['Duração média da visualização'] = conteudo['Duração média da visualização'].apply(lambda x: pd.to_timedelta(x, unit = 's'))
conteudo['Duração média da visualização'] = pd.to_timedelta(conteudo['Duração média da visualização']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(conteudo['Duração média da visualização']).dt.components['seconds'].astype(str).str.zfill(2)
conteudo['Minutagem média'] = conteudo['Minutagem média'] / 60
conteudo_minutagem = conteudo['Minutagem média'].mean()
conteudo = conteudo.reset_index()
conteudo = pd.merge(conteudo, videos_publicados[['Data', 'Vídeos Publicados']], on = 'Data', how = 'left')
conteudo_pubs = conteudo['Vídeos Publicados'].mean()
cols = conteudo.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[1:-1]
conteudo = conteudo[cols]
conteudo.sort_values(by = 'Data', inplace = True)
conteudo.rename(columns=colunas_conteudo, inplace=True)
conteudo.set_index('Data', inplace = True)
conteudo = conteudo.T
conteudo.insert(0, 'Média', [conteudo_pubs,conteudo_impressoes, conteudo_visualizacoes, conteudo_ctr, conteudo_tempo_assistido, conteudo_porcentagem_assistida, conteudo_rpm, conteudo_nao_gostei, conteudo_inscritos, conteudo_minutagem])
conteudo.reset_index(inplace = True)
conteudo.rename(columns={'index':'Analise de Conteúdo Novo'}, inplace = True)
conteudo.fillna(0, inplace=True)
conteudo


Data,Analise de Conteúdo Novo,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Vídeos Publicados,79.833333,105,82,68,73,71,80
1,Impressões Novo,1357039.333333,1515576,1296534,1234480,1548743,1263467,1283436
2,Visualizações Novo,146381.333333,156986,140623,131540,160148,138700,150291
3,CTR (%) Novo,0.053231,0.048477,0.050999,0.054779,0.055122,0.05619,0.053818
4,Tempo Médio Assistido Novo,14:06,12:53,13:11,14:19,15:13,15:07,13:53
5,Porcentagem Média Assistida Novo,0.171024,0.19473,0.157616,0.161397,0.166366,0.17392,0.172116
6,RPM (USD) Novo,1.023755,0.915818,1.275164,0.964667,0.823649,1.093314,1.06992
7,"Porcentagem de ""Não Gostei"" Novo",0.00047,0.000285,0.000442,0.000391,0.000479,0.00064,0.000582
8,Saldo de Inscritos Novo,393.0,419,314,382,398,367,478
9,Tamanho médio do vídeo (min) Novo,82.986214,66.238073,83.725998,88.754746,91.525521,86.949891,80.723058


In [294]:
conteudo_temp = conteudo.copy()
conteudo_temp.set_index('Analise de Conteúdo Novo', inplace = True)
conteudo_temp.loc['Tempo Médio Assistido Novo'] = conteudo_temp.loc['Tempo Médio Assistido Novo'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))
conteudo_desvio = conteudo_temp.copy()
conteudo_desvio.iloc[:, 1:] = conteudo_desvio.iloc[:, 1:].div(conteudo_desvio['Média'], axis=0) - 1
conteudo_desvio.drop('Média', axis=1, inplace=True)
conteudo_desvio.reset_index(drop=True, inplace=True)
conteudo_desvio.insert(0, 'Análise de Conteúdo Novo', conteudo['Analise de Conteúdo Novo'])
conteudo_desvio.fillna(0, inplace=True)
conteudo_desvio

Data,Análise de Conteúdo Novo,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Vídeos Publicados,0.31524,0.02714,-0.148225,-0.085595,-0.110647,0.002088
1,Impressões Novo,0.116825,-0.044586,-0.090314,0.141266,-0.068953,-0.054238
2,Visualizações Novo,0.072445,-0.039338,-0.101388,0.094047,-0.052475,0.026709
3,CTR (%) Novo,-0.089303,-0.041931,0.029092,0.035526,0.055594,0.011022
4,Tempo Médio Assistido Novo,-0.086288,-0.065012,0.015366,0.079196,0.072104,-0.015366
5,Porcentagem Média Assistida Novo,0.138614,-0.0784,-0.056291,-0.027238,0.016931,0.006385
6,RPM (USD) Novo,-0.105433,0.245575,-0.057718,-0.195463,0.067944,0.045093
7,"Porcentagem de ""Não Gostei"" Novo",-0.393262,-0.058861,-0.167364,0.019289,0.361318,0.238879
8,Saldo de Inscritos Novo,0.066158,-0.201018,-0.02799,0.012723,-0.066158,0.216285
9,Tamanho médio do vídeo (min) Novo,-0.201818,0.008915,0.069512,0.1029,0.047763,-0.027271


In [295]:
#Formatação de porcentagens conteúdo
conteudo.set_index('Analise de Conteúdo Novo', inplace=True)
conteudo.loc['CTR (%) Novo'] = conteudo.loc['CTR (%) Novo'].map(format_as_percentage)
conteudo.loc['Porcentagem de "Não Gostei" Novo'] = conteudo.loc['Porcentagem de "Não Gostei" Novo'].map(format_as_percentage)
conteudo.loc['Porcentagem Média Assistida Novo'] = conteudo.loc['Porcentagem Média Assistida Novo'].map(format_as_percentage)
conteudo.reset_index(inplace=True)
conteudo = conteudo.astype(str)

#Formatação de porcentagens desvio
conteudo_desvio.iloc[:, 1:] = conteudo_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
conteudo_desvio_formatado = conteudo_desvio.iloc[:, 1:].map(format_as_percentage)
conteudo_desvio_formatado = pd.concat([conteudo_desvio['Análise de Conteúdo Novo'], conteudo_desvio_formatado], axis=1)
conteudo_desvio_formatado = conteudo_desvio_formatado.astype(str)

# Tabela de Receita

In [296]:
receita = data_1[['Data', 'Receita estimada (USD)']]
receita = pd.merge(receita, conteudo_1[['Data', 'Receita estimada (USD)']], on = 'Data', how = 'left')
receita['Conteúdo Velho'] = receita['Receita estimada (USD)_x'] - receita['Receita estimada (USD)_y']

receita_total = receita['Receita estimada (USD)_x'].mean()
receita_novo = receita['Receita estimada (USD)_y'].mean()
receita_velho = receita['Conteúdo Velho'].mean()


receita.sort_values(by = 'Data', inplace = True)
receita.rename(columns=colunas_receita, inplace=True)
receita.set_index('Data', inplace = True)
receita = receita[colunas_receita_ordenada]
receita.insert(3,'Receita % Novo',(receita['Receita Conteúdo novo (Soma)']/receita['Receita Total']))
receita.insert(4,'Receita % Velho',(receita['Receita Conteúdo velho (Soma)']/receita['Receita Total']))

receita_percent_novo = receita['Receita % Novo'].mean()
receita_percent_velho = receita['Receita % Velho'].mean()

receita = receita.T
receita.insert(0, 'Média', [receita_novo, receita_velho, receita_total, receita_percent_novo, receita_percent_velho])
receita.reset_index(inplace = True)
receita.rename(columns={'index':'Receita (USD)'}, inplace = True)
receita.fillna(0, inplace=True)
receita

Data,Receita (USD),Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Receita Conteúdo novo (Soma),165.179000,124.314000,216.971000,146.189000,156.305000,164.417000,182.878000
1,Receita Conteúdo velho (Soma),18.720167,23.717000,26.883000,11.943000,15.584000,28.370000,5.824000
2,Receita Total,183.899167,148.031000,243.854000,158.132000,171.889000,192.787000,188.702000
3,Receita % Novo,0.897555,0.839784,0.889758,0.924474,0.909337,0.852843,0.969137
4,Receita % Velho,0.102445,0.160216,0.110242,0.075526,0.090663,0.147157,0.030863


In [297]:
receita_desvio = receita.copy()
receita_desvio.iloc[:, 1:] = (receita_desvio.iloc[:, 1:].div(receita_desvio['Média'], axis=0) - 1)
receita_desvio.drop('Média', axis=1, inplace=True)
receita_desvio.fillna(0, inplace=True)
receita_desvio

Data,Receita (USD),2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Receita Conteúdo novo (Soma),-0.247398,0.313551,-0.114966,-0.053724,-0.004613,0.107150
1,Receita Conteúdo velho (Soma),0.266922,0.436045,-0.362025,-0.167529,0.515478,-0.688892
2,Receita Total,-0.195043,0.326020,-0.140116,-0.065308,0.048330,0.026117
3,Receita % Novo,-0.064366,-0.008688,0.029992,0.013126,-0.049816,0.079751
4,Receita % Velho,0.563931,0.076115,-0.262768,-0.115004,0.436456,-0.698730


In [298]:
receita.set_index('Receita (USD)', inplace=True)
receita.loc['Receita % Novo'] = receita.loc['Receita % Novo'].map(format_as_percentage)
receita.loc['Receita % Velho'] = receita.loc['Receita % Velho'].map(format_as_percentage)
receita.reset_index(inplace=True)
receita = receita.astype(str)

#Formatação de porcentagens desvio
receita_desvio.iloc[:, 1:] = receita_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
receita_desvio_formatado = receita_desvio.iloc[:, 1:].map(format_as_percentage)
receita_desvio_formatado = pd.concat([receita_desvio['Receita (USD)'], receita_desvio_formatado], axis=1)
receita_desvio_formatado = receita_desvio_formatado.astype(str)

/var/folders/mb/gdj6y8k92lx9_njl612qh22m0000gn/T/ipykernel_55089/4211859304.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '89.76%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  receita.loc['Receita % Novo'] = receita.loc['Receita % Novo'].map(format_as_percentage)
/var/folders/mb/gdj6y8k92lx9_njl612qh22m0000gn/T/ipykernel_55089/4211859304.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '83.98%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  receita.loc['Receita % Novo'] = receita.loc['Receita % Novo'].map(format_as_percentage)
/var/folders/mb/gdj6y8k92lx9_njl612qh22m0000gn/T/ipykernel_55089/4211859304.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '88.98%' has dtype incompat

# Tabela de Shorts

In [299]:
shorts = pd.read_csv(f'dados_full/{artista}/shorts.csv')
shorts = shorts.drop(shorts.index[0])
shorts['Data'] = pd.to_datetime(shorts['Data']).dt.to_period('M')
shorts['Duração média da visualização'] = pd.to_timedelta(shorts['Duração média da visualização']).apply(lambda x: x.total_seconds())
shorts['Minutagem média'] = (shorts['Duração média da visualização'] * 100) / shorts['Porcentagem visualizada média (%)']
shorts['Taxa de cliques de impressões (%)'] = shorts['Taxa de cliques de impressões (%)'] / 100
shorts = shorts[['Data', 'Vídeos publicados', 'Impressões', 'Visualizações', 'Taxa de cliques de impressões (%)', 'RPM (USD)', 'Duração média da visualização', 'Minutagem média', 'Inscritos']]

shorts_publicados = shorts['Vídeos publicados'].mean()
shorts_impressões = shorts['Impressões'].mean()
shorts_views = shorts['Visualizações'].mean()
shorts_ctr = shorts['Taxa de cliques de impressões (%)'].mean()
shorts_rpm = shorts['RPM (USD)'].mean()
shorts_tempo_assistido = shorts['Duração média da visualização'].mean()
shorts_tempo_assistido = '{:02d}:{:02d}'.format(int(shorts_tempo_assistido // 60), int(shorts_tempo_assistido % 60))
shorts_minutagem = shorts['Minutagem média'].mean()
shorts_minutagem = '{:02d}:{:02d}'.format(int(shorts_minutagem // 60), int(shorts_minutagem % 60))
shorts_inscritos = shorts['Inscritos'].mean()

shorts['Duração média da visualização'] = shorts['Duração média da visualização'].apply(lambda x: pd.to_timedelta(x, unit = 's'))
shorts['Duração média da visualização'] = pd.to_timedelta(shorts['Duração média da visualização']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(shorts['Duração média da visualização']).dt.components['seconds'].astype(str).str.zfill(2)
shorts['Minutagem média'] = shorts['Minutagem média'].apply(lambda x: pd.to_timedelta(x, unit = 's'))
shorts['Minutagem média'] = pd.to_timedelta(shorts['Minutagem média']).dt.components['minutes'].astype(str).str.zfill(2) + ':' + pd.to_timedelta(shorts['Minutagem média']).dt.components['seconds'].astype(str).str.zfill(2)
shorts.sort_values(by='Data', inplace=True, ascending=True)
shorts.rename(columns = colunas_shorts, inplace = True)
# shorts['Tamanho de Vídeo Shorts'].loc[[6,5,4,3,2]]= '00:00'
# shorts['Tamanho de Vídeo Shorts'].loc[[1]]= '00:36'
shorts.set_index('Data', inplace = True)
shorts = shorts.T
shorts.insert(0, 'Média', [shorts_publicados, shorts_impressões, shorts_views, shorts_ctr, shorts_rpm, shorts_tempo_assistido, shorts_minutagem, shorts_inscritos])
shorts.reset_index(inplace=True)
shorts.rename(columns={'index':'Shorts'}, inplace=True)
shorts.fillna(0, inplace=True)
shorts

Data,Shorts,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de Shorts,5.5,6,9,7,8,1,2
1,Impressões Shorts,157572.833333,164779,158245,163488,162747,151293,144885
2,Visualizações Shorts,13343.666667,18693,19167,13499,15430,6996,6277
3,Taxa de cliques de impressões (%),0.021717,0.0235,0.0227,0.024,0.022,0.0186,0.0195
4,RPM Shorts,0.028167,0.024,0.026,0.027,0.029,0.032,0.031
5,Tempo Médio Assistido Shorts,00:42,00:38,00:36,00:46,00:39,00:44,00:50
6,Tamanho de Vídeo Shorts,00:43,00:43,00:40,00:45,00:42,00:45,00:46
7,Saldo de Inscritos Shorts,49.666667,69,67,57,50,33,22


In [300]:
shorts_temp = shorts.copy()
shorts_temp.set_index('Shorts', inplace=True)
shorts_temp.loc['Tamanho de Vídeo Shorts'] = shorts_temp.loc['Tamanho de Vídeo Shorts'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))
shorts_temp.loc['Tempo Médio Assistido Shorts'] = shorts_temp.loc['Tempo Médio Assistido Shorts'].apply(lambda x: sum(int(i) * 60 ** index for index, i in enumerate(reversed(x.split(':')))))

shorts_desvio = shorts_temp.copy()
shorts_desvio.iloc[:, 1:] = (shorts_desvio.iloc[:, 1:].div(shorts_desvio['Média'], axis=0) - 1)

shorts_desvio.drop('Média', axis=1, inplace=True)
shorts_desvio.reset_index(drop=True, inplace=True)
shorts_desvio.insert(0, 'Shorts', shorts['Shorts'])
shorts_desvio.fillna(0, inplace=True)
shorts.fillna(0, inplace=True)
shorts_desvio

Data,Shorts,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de Shorts,0.090909,0.636364,0.272727,0.454545,-0.818182,-0.636364
1,Impressões Shorts,0.045732,0.004266,0.037539,0.032837,-0.039854,-0.08052
2,Visualizações Shorts,0.400889,0.436412,0.011641,0.156354,-0.475706,-0.52959
3,Taxa de cliques de impressões (%),0.082118,0.04528,0.105142,0.013047,-0.143515,-0.102072
4,RPM Shorts,-0.147929,-0.076923,-0.04142,0.029586,0.136095,0.100592
5,Tempo Médio Assistido Shorts,-0.095238,-0.142857,0.095238,-0.071429,0.047619,0.190476
6,Tamanho de Vídeo Shorts,0.0,-0.069767,0.046512,-0.023256,0.046512,0.069767
7,Saldo de Inscritos Shorts,0.389262,0.348993,0.147651,0.006711,-0.33557,-0.557047


In [301]:
shorts.set_index('Shorts', inplace=True)
shorts.loc['Taxa de cliques de impressões (%)'] = shorts.loc['Taxa de cliques de impressões (%)'].map(format_as_percentage)
shorts.reset_index(inplace=True)
shorts = shorts.astype(str)

#Formatação de porcentagens desvio
shorts_desvio.iloc[:, 1:] = shorts_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
shorts_desvio_formatado = shorts_desvio.iloc[:, 1:].map(format_as_percentage)
shorts_desvio_formatado = pd.concat([shorts_desvio['Shorts'], shorts_desvio_formatado], axis=1)
shorts_desvio_formatado = shorts_desvio_formatado.astype(str)

# Tabela de Origem de Tráfego

In [302]:
origem = pd.read_csv(f'dados_full/{artista}/origem.csv')
colunas_ori = origem['Origem do tráfego'].unique().tolist()
colunas_origem = ['Data'] + colunas_ori

origem_pivot = origem.pivot_table(index='Data', columns='Origem do tráfego', values='Visualizações', aggfunc='first').reset_index()
origem_pivot.columns.name = None

colunas_desejadas = [
    'Recursos de navegação',
    'Vídeos sugeridos',
    'Páginas do canal',
    'Externa',
    'Notificações',
    'Pesquisa do YouTube',
    'Playlists',
    'Publicidade no YouTube'
]

# Usar copy() aqui para evitar SettingWithCopyWarning
origem_filtrado = origem_pivot[['Data'] + colunas_desejadas].copy()
origem_filtrado['Outros'] = origem_pivot.drop(['Data'] + colunas_desejadas, axis=1).sum(axis=1)
origem_filtrado = origem_filtrado.set_index('Data')
origem = origem_filtrado.copy()
origem

meses = origem_filtrado.index.tolist()
soma_origem = origem_filtrado.iloc[:, 0:].sum(axis=1)
soma_origem = pd.DataFrame(soma_origem)
soma_origem = soma_origem.rename(columns = {0:'Total'})
soma_origem.insert(0, 'Data', meses)
soma_origem.set_index('Data', inplace = True)
soma_origem

recursos_org = origem['Recursos de navegação'].mean()
sugeri_org = origem['Vídeos sugeridos'].mean()
pagina_org = origem['Páginas do canal'].mean()
extern_org = origem['Externa'].mean()
notifi_org = origem['Notificações'].mean()
pesquisa_org = origem['Pesquisa do YouTube'].mean()
playlist_org = origem['Playlists'].mean()
publi_org = origem['Publicidade no YouTube'].mean()
outros_org = origem['Outros'].mean()

origem.rename(columns={'Publicidade no YouTube':'Tráfego Pago'}, inplace=True)

origem = origem.T
origem.insert(0, 'Média', [recursos_org, sugeri_org, pagina_org, extern_org, notifi_org, pesquisa_org, playlist_org, publi_org,outros_org,])
origem.reset_index(inplace=True)
origem.rename(columns={'index':'Origem de Tráfego'}, inplace=True)
origem.fillna(0, inplace=True)
# origem['Média'] = origem['Média'].apply(lambda x: f'{x:.2f}')
origem

Data,Origem de Tráfego,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Recursos de navegação,64692.833333,67912,64588,58324,71556,62732,63045
1,Vídeos sugeridos,17261.833333,20563,16109,14280,17713,18680,16226
2,Páginas do canal,15605.166667,14127,15378,14812,19856,15280,14178
3,Externa,15809.833333,18856,18849,12775,14647,14461,15271
4,Notificações,5368.666667,6640,5420,5445,5110,4383,5214
5,Pesquisa do YouTube,37782.500000,38952,36963,33888,37940,39809,39143
6,Playlists,1568.500000,1588,1390,1525,1795,1411,1702
7,Tráfego Pago,0.000000,0,0,0,0,0,0
8,Outros,11077.000000,11544,10655,9426,12023,11142,11672


In [303]:
origem_desvio = origem.copy()
origem_desvio.set_index('Origem de Tráfego', inplace=True)
origem_desvio.iloc[:, 1:] = (origem_desvio.iloc[:, 1:].div(origem_desvio['Média'], axis=0) - 1)
origem_desvio.drop('Média', axis=1, inplace=True)
origem_desvio.reset_index(drop=True, inplace=True)
origem_desvio.insert(0, 'Origem de Tráfego', origem['Origem de Tráfego'])
origem_desvio.fillna(0, inplace=True)
origem_desvio

Data,Origem de Tráfego,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Recursos de navegação,0.049761,-0.001620,-0.098447,0.106089,-0.030310,-0.025472
1,Vídeos sugeridos,0.191241,-0.066785,-0.172741,0.026137,0.082156,-0.060007
2,Páginas do canal,-0.094723,-0.014557,-0.050827,0.272399,-0.020837,-0.091455
3,Externa,0.192675,0.192233,-0.191959,-0.073551,-0.085316,-0.034082
4,Notificações,0.236806,0.009562,0.014218,-0.048181,-0.183596,-0.028809
5,Pesquisa do YouTube,0.030953,-0.021690,-0.103077,0.004169,0.053636,0.036009
6,Playlists,0.012432,-0.113803,-0.027734,0.144405,-0.100414,0.085113
7,Tráfego Pago,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Outros,0.042159,-0.038097,-0.149048,0.085402,0.005868,0.053715


In [304]:
origem.iloc[:, 1:] = origem.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
origem['Média'] = origem['Média'].apply(lambda x: f'{x:.2f}')
perc_origem_formatado = origem.astype(str)

#Formatação de porcentagens desvio
origem_desvio.iloc[:, 1:] = origem_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
origem_desvio_formatado = origem_desvio.iloc[:, 1:].map(format_as_percentage)
origem_desvio_formatado = pd.concat([origem_desvio['Origem de Tráfego'], origem_desvio_formatado], axis=1)
origem_desvio_formatado = origem_desvio_formatado.astype(str)


# Comunidade

In [305]:
data_comu_2 = pd.read_csv('dados_full/'+artista+'/total.csv')
data_comu_2 = data_comu_2.drop(data_comu_2.index[0])
data_comu_2['Data'] = pd.to_datetime(data_comu_2['Data'])
data_comu_2['Mes'] = data_comu_2['Data'].dt.to_period('M')
data_comu_2.sort_values(by='Mes', inplace=True)
data_comu_2 = data_comu_2.drop('Data', axis=1)
colunas = data_comu_2.columns.tolist()
colunas = [colunas[-1]] + colunas[:-1]
data_comu_2 = data_comu_2[colunas]
data_comu_2


,Mes,"Média de ""Gostei"" da postagem (%)","""Gostei"" da postagem",Impressões da postagem,"Marcações ""Não gostei""",RPM (USD),Taxa de cliques de impressões (%),Impressões,Vídeos publicados,Inscritos,Porcentagem visualizada média (%),Visualizações,Tempo de exibição (horas),Duração média da visualização,Receita estimada (USD)
1,2023-11,3.78,587,14246,99,0.954,5.71,2771208,179,1226,20.69,265748,75462.2565,0:17:02,253.494
3,2023-12,2.65,330,10832,96,1.331,5.91,2498095,144,1048,19.82,250279,68223.5081,0:16:21,333.112
6,2024-01,1.86,174,8700,95,1.064,5.83,2423573,119,1115,21.37,230294,67122.0152,0:17:29,244.967
2,2024-02,3.38,101,2545,112,0.977,5.92,2719218,119,1287,19.90,262630,78010.5307,0:17:49,256.483
4,2024-03,2.43,34,1276,97,1.197,5.66,2721410,112,1400,21.24,252927,75664.4984,0:17:57,302.690
5,2024-04,2.05,61,2392,104,1.134,6.36,2314296,107,1340,21.40,239658,74186.2674,0:18:34,271.820


In [306]:
data_comu = pd.read_csv('dados_full/'+artista+'/comunidade.csv')
data_comu.rename(columns=colunas_comunidade,inplace=True)
data_comu["Horário de publicação da postagem"] = pd.to_datetime(data_comu['Horário de publicação da postagem'], format="%b %d, %Y", errors='coerce')
data_comu.sort_values('Horário de publicação da postagem',ascending=False,inplace=True)
auxMeses_comu = mes.month+6
if auxMeses_comu > 12:
    auxMeses_comu = auxMeses_comu - 12 
comuCount = data_comu.loc[(data_comu["Horário de publicação da postagem"] > '2023-06')]
comuCount = comuCount.groupby([comuCount['Horário de publicação da postagem'].dt.month], sort=False).count()
df_graf_com = data_comu_2[['Mes','Impressões da postagem','"Gostei" da postagem','Média de "Gostei" da postagem (%)']].set_index('Mes').copy()
comuCount = comuCount[comuCount.index.isin(meses)]
auxComu = 1
auxList = []
for x in range(6):
    if len(comuCount.index) < auxComu:
        auxList.append(meses[x])
    else:
        if comuCount.index[-auxComu] == meses[x]:
            auxComu = auxComu + 1
        else:
            auxList.append(meses[x])
for x in auxList:
    comuCount.loc[x] = 0    
df_graf_com['Publicações na Comunidade'] = [comuCount['Postar'].loc[(comuCount.index == meses[-6])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-5])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-4])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-3])].item(),
                                            comuCount['Postar'].loc[(comuCount.index == meses[-2])].item(),
                                           comuCount['Postar'].loc[(comuCount.index == meses[-1])].item()]
df_graf_com['Média de Impressões por postagem'] =  (df_graf_com['Impressões da postagem']/df_graf_com['Publicações na Comunidade'])
df_graf_com['Média de Impressões por postagem'] = df_graf_com['Média de Impressões por postagem'].replace([np.inf, -np.inf], 0)
df_graf_com = df_graf_com.fillna(0)
df_graf_com['Média de Impressões por postagem'] = df_graf_com['Média de Impressões por postagem'].astype(int)
df_graf_com = df_graf_com[['Publicações na Comunidade','Impressões da postagem','Média de Impressões por postagem','"Gostei" da postagem','Média de "Gostei" da postagem (%)']]

pubs_com = df_graf_com['Publicações na Comunidade'].mean()
impres_com = df_graf_com['Impressões da postagem'].mean()
impress_mean_com = df_graf_com['Média de Impressões por postagem'].mean()
likes_com = df_graf_com['"Gostei" da postagem'].mean()
likes_mean_com = df_graf_com['Média de "Gostei" da postagem (%)'].mean()


df_graf_com = df_graf_com.rename(columns={"Impressões da postagem": "Total de Impressões na Comunidade"})
df_graf_com = df_graf_com.T

df_graf_com.insert(0, 'Média', [pubs_com,impres_com,impress_mean_com,likes_com,likes_mean_com])
df_graf_com.reset_index(inplace=True)
df_graf_com.rename(columns={'index':'Comunidade'}, inplace=True)
df_graf_com

Mes,Comunidade,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Publicações na Comunidade,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
1,Total de Impressões na Comunidade,6665.166667,14246.00,10832.00,8700.00,2545.00,1276.00,2392.00
2,Média de Impressões por postagem,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
3,"""Gostei"" da postagem",214.500000,587.00,330.00,174.00,101.00,34.00,61.00
4,"Média de ""Gostei"" da postagem (%)",2.691667,3.78,2.65,1.86,3.38,2.43,2.05


In [307]:
comu_desvio = df_graf_com.copy()
comu_desvio.set_index('Comunidade', inplace=True)
comu_desvio.iloc[:, 1:] = (comu_desvio.iloc[:, 1:].div(comu_desvio['Média'], axis=0) - 1)
comu_desvio.drop('Média', axis=1, inplace=True)
comu_desvio.reset_index(drop=True, inplace=True)
comu_desvio.insert(0, 'Comunidade', df_graf_com['Comunidade'])
comu_desvio.fillna(0, inplace=True)
comu_desvio

Mes,Comunidade,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Publicações na Comunidade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Total de Impressões na Comunidade,1.137381,0.625166,0.305294,-0.618164,-0.808557,-0.641119
2,Média de Impressões por postagem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"""Gostei"" da postagem",1.736597,0.538462,-0.188811,-0.529138,-0.841492,-0.715618
4,"Média de ""Gostei"" da postagem (%)",0.404334,-0.015480,-0.308978,0.255728,-0.097214,-0.238390


In [308]:
df_graf_com.iloc[:, 1:] = df_graf_com.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
df_graf_com['Média'] = df_graf_com['Média'].apply(lambda x: f'{x:.2f}')
df_graf_com_formatado = df_graf_com.astype(str)

#Formatação de porcentagens desvio
comu_desvio.iloc[:, 1:] = comu_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
comu_desvio_formatado = comu_desvio.iloc[:, 1:].map(format_as_percentage)
comu_desvio_formatado = pd.concat([comu_desvio['Comunidade'], comu_desvio_formatado], axis=1)
comu_desvio_formatado = comu_desvio_formatado.astype(str)

# Videos Publicados

In [309]:
vods = pd.read_csv(f'dados_full/{artista}/subs.csv')
vods = vods.drop(vods.index[0])
vods['Data'] = pd.to_datetime(vods['Data'])
vods['Data'] = vods['Data'].dt.to_period('M')
vods.sort_values(by='Data', inplace=True)
vods = vods[['Data', 'Vídeos publicados']]
vods['Vídeos publicados'] = vods['Vídeos publicados'].round(0)
vods.rename(columns = {'Vídeos publicados': "Número de VOD's"}, inplace=True)
vods.fillna(0,inplace=True)

p_shorts = pd.read_csv(f'dados_full/{artista}/shorts.csv')
p_shorts = p_shorts.drop(p_shorts.index[0])
p_shorts['Data'] = pd.to_datetime(p_shorts['Data'])
p_shorts['Data'] = p_shorts['Data'].dt.to_period('M')
p_shorts.sort_values(by='Data', inplace=True)
p_shorts = p_shorts[['Data', 'Vídeos publicados']]
p_shorts.rename(columns = {'Vídeos publicados': 'Número de Shorts'}, inplace=True)
p_shorts.fillna(0, inplace=True)

p_lives = pd.read_csv(f'dados_full/{artista}/lives.csv')
p_lives = p_lives.drop(p_lives.index[0])
p_lives['Data'] = pd.to_datetime(p_lives['Data'])
p_lives['Data'] = p_lives['Data'].dt.to_period('M')
p_lives.sort_values(by='Data', inplace=True)
p_lives = p_lives[['Data', 'Vídeos publicados']]
p_lives.rename(columns = {'Vídeos publicados': 'Número de Lives'}, inplace=True)
p_lives.fillna(0, inplace=True)

publicados = pd.merge(vods, p_shorts[['Data', 'Número de Shorts']], on='Data', how='left')
publicados = pd.merge(publicados, p_lives[['Data', 'Número de Lives']], on='Data', how='left')
publicados.set_index('Data', inplace=True)

p_vod_mean = publicados["Número de VOD's"].mean()
p_shorts_mean = publicados["Número de Shorts"].mean()
p_lives_mean = publicados["Número de Lives"].mean()

publicados = publicados.T
publicados.insert(0,'Média', [p_vod_mean,p_shorts_mean,p_lives_mean])
publicados.reset_index(inplace=True)
publicados.rename(columns={'index':'Vídeos Publicados'}, inplace=True)
publicados.fillna(0, inplace = True)
publicados


Data,Vídeos Publicados,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de VOD's,22.666667,42,27,25,20,16,6
1,Número de Shorts,5.500000,6,9,7,8,1,2
2,Número de Lives,101.833333,131,108,87,91,95,99


In [310]:
pubs_desvio = publicados.copy()
pubs_desvio.set_index('Vídeos Publicados', inplace=True)
pubs_desvio.iloc[:, 1:] = (pubs_desvio.iloc[:, 1:].div(pubs_desvio['Média'], axis=0) - 1)
pubs_desvio.drop('Média', axis=1, inplace=True)
pubs_desvio.reset_index(drop=True, inplace=True)
pubs_desvio.insert(0, 'Vídeos Publicados', publicados['Vídeos Publicados'])
pubs_desvio.fillna(0, inplace=True)
pubs_desvio

Data,Vídeos Publicados,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Número de VOD's,0.852941,0.191176,0.102941,-0.117647,-0.294118,-0.735294
1,Número de Shorts,0.090909,0.636364,0.272727,0.454545,-0.818182,-0.636364
2,Número de Lives,0.286416,0.060556,-0.145663,-0.106383,-0.067103,-0.027823


In [311]:
publicados.iloc[:, 1:] = publicados.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
publicados['Média'] = publicados['Média'].apply(lambda x: f'{x:.2f}')
publicados_formatado = publicados.astype(str)

#Formatação de porcentagens desvio
pubs_desvio.iloc[:, 1:] = pubs_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
pubs_desvio_formatado = pubs_desvio.iloc[:, 1:].map(format_as_percentage)
pubs_desvio_formatado = pd.concat([pubs_desvio['Vídeos Publicados'], pubs_desvio_formatado], axis=1)
pubs_desvio_formatado = pubs_desvio_formatado.astype(str)

# WatchTime

In [312]:
watch = pd.read_csv(f'dados_full/{artista}/total.csv')
watch = watch.drop(watch.index[0])
watch = watch[['Data', 'Tempo de exibição (horas)']].round(2)
watch.sort_values(by='Data', inplace=True)
watch.rename(columns={'Tempo de exibição (horas)':'WatchTime (horas)'}, inplace=True)
watch.set_index('Data', inplace=True)

watch_mean = watch['WatchTime (horas)'].mean()

watch = watch.T
watch.insert(0, 'Média', watch_mean)
watch.reset_index(inplace=True)
watch.rename(columns={'index':'WatchTime'}, inplace=True)
watch

Data,WatchTime,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,WatchTime (horas),73111.515,75462.26,68223.51,67122.02,78010.53,75664.5,74186.27


In [313]:
watch_desvio = watch.copy()
watch_desvio.set_index('WatchTime', inplace=True)
watch_desvio.iloc[:, 1:] = (watch_desvio.iloc[:, 1:].div(watch_desvio['Média'], axis=0) - 1)
watch_desvio.drop('Média', axis=1, inplace=True)
watch_desvio.reset_index(drop=True, inplace=True)
watch_desvio.insert(0, 'WatchTime', watch['WatchTime'])
watch_desvio.fillna(0, inplace=True)
watch_desvio

Data,WatchTime,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,WatchTime (horas),0.032153,-0.066857,-0.081923,0.067007,0.034919,0.0147


In [314]:
watch.iloc[:, 1:] = watch.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
watch['Média'] = watch['Média'].apply(lambda x: f'{x:.2f}')
watch_formatado = watch.astype(str)

#Formatação de porcentagens desvio
watch_desvio.iloc[:, 1:] = watch_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
watch_desvio_formatado = watch_desvio.iloc[:, 1:].map(format_as_percentage)
watch_desvio_formatado = pd.concat([watch_desvio['WatchTime'], watch_desvio_formatado], axis=1)
watch_desvio_formatado = watch_desvio_formatado.astype(str)

# Inscritos

In [315]:
insc_vods = pd.read_csv(f'dados_full/{artista}/subs.csv')
insc_vods = insc_vods.drop(insc_vods.index[0])
insc_vods['Data'] = pd.to_datetime(insc_vods['Data'])
insc_vods['Data'] = insc_vods['Data'].dt.to_period('M')
insc_vods.sort_values(by='Data', inplace=True)
insc_vods = insc_vods[['Data', 'Inscritos']]
insc_vods['Inscritos'] = insc_vods['Inscritos'].round(0)
insc_vods.rename(columns = {'Inscritos': "Inscritos por VOD's"}, inplace=True)

insc_shorts = pd.read_csv(f'dados_full/{artista}/shorts.csv')
insc_shorts = insc_shorts.drop(insc_shorts.index[0])
insc_shorts['Data'] = pd.to_datetime(insc_shorts['Data'])
insc_shorts['Data'] = insc_shorts['Data'].dt.to_period('M')
insc_shorts.sort_values(by='Data', inplace=True)
insc_shorts = insc_shorts[['Data', 'Inscritos']]
insc_shorts.rename(columns = {'Inscritos': 'Inscritos por Shorts'}, inplace=True)
insc_shorts.fillna(0, inplace=True)

insc_lives = pd.read_csv(f'dados_full/{artista}/lives.csv')
insc_lives = insc_lives.drop(insc_lives.index[0])
insc_lives['Data'] = pd.to_datetime(insc_lives['Data'])
insc_lives['Data'] = insc_lives['Data'].dt.to_period('M')
insc_lives.sort_values(by='Data', inplace=True)
insc_lives = insc_lives[['Data', 'Inscritos']]
insc_lives.rename(columns = {'Inscritos': 'Inscritos por Lives'}, inplace=True)
insc_lives.fillna(0, inplace=True)

inscritos = pd.merge(insc_vods, insc_shorts[['Data', 'Inscritos por Shorts']], on='Data', how='left')
inscritos = pd.merge(inscritos, insc_lives[['Data', 'Inscritos por Lives']], on='Data', how='left')
inscritos.set_index('Data', inplace=True)

insc_vods_mean = inscritos["Inscritos por VOD's"].mean()
insc_shorts_mean = inscritos["Inscritos por Shorts"].mean()
insc_lives_mean = inscritos["Inscritos por Lives"].mean()

inscritos = inscritos.T
inscritos.insert(0,'Média',[insc_vods_mean, insc_shorts_mean, insc_lives_mean])
inscritos.reset_index(inplace=True)
inscritos.rename(columns={'index':'Inscritos'}, inplace=True)
inscritos.fillna(0, inplace=True)
inscritos

Data,Inscritos,Média,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Inscritos por VOD's,363.333333,335,281,371,369,448,376
1,Inscritos por Shorts,49.666667,69,67,57,50,33,22
2,Inscritos por Lives,506.166667,526,437,472,513,520,569


In [316]:
insc_desvio = inscritos.copy()
insc_desvio.set_index('Inscritos', inplace=True)
insc_desvio.iloc[:, 1:] = (insc_desvio.iloc[:, 1:].div(insc_desvio['Média'], axis=0) - 1)
insc_desvio.drop('Média', axis=1, inplace=True)
insc_desvio.reset_index(drop=True, inplace=True)
insc_desvio.insert(0, 'Inscritos', inscritos['Inscritos'])
insc_desvio.fillna(0, inplace=True)
insc_desvio

Data,Inscritos,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04
0,Inscritos por VOD's,-0.077982,-0.226606,0.021101,0.015596,0.233028,0.034862
1,Inscritos por Shorts,0.389262,0.348993,0.147651,0.006711,-0.335570,-0.557047
2,Inscritos por Lives,0.039183,-0.136648,-0.067501,0.013500,0.027330,0.124136


In [317]:
inscritos.iloc[:, 1:] = inscritos.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
inscritos['Média'] = inscritos['Média'].apply(lambda x: f'{x:.2f}')
inscritos_formatado = inscritos.astype(str)

#Formatação de porcentagens desvio
insc_desvio.iloc[:, 1:] = insc_desvio.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
insc_desvio_formatado = insc_desvio.iloc[:, 1:].map(format_as_percentage)
insc_desvio_formatado = pd.concat([insc_desvio['Inscritos'], insc_desvio_formatado], axis=1)
insc_desvio_formatado = insc_desvio_formatado.astype(str)

# Montagem do arquivo

In [318]:
with pd.ExcelWriter(f'exports_tabelas/Monthly Abril 2024 ({artista})_Lives.xlsx') as writer:
    
    data.to_excel(writer, sheet_name='Resultado', index=False)

    num_rows_df1 = data.shape[0]

    conteudo.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1+1, index=False)

    num_rows_df2 = conteudo.shape[0]

    receita.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + 2, index=False)

    num_rows_df3 = receita.shape[0]

    shorts.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + 3, index=False)

    num_rows_df4 = shorts.shape[0]

    perc_origem_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + 4, index=False)

    num_rows_df5 = perc_origem_formatado.shape[0]
    
    df_graf_com_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + 5, index=False)

    num_rows_df6 = df_graf_com_formatado.shape[0]

    publicados_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + num_rows_df6 + 6, index=False)

    num_rows_df7 = publicados_formatado.shape[0]

    watch_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + num_rows_df6 + num_rows_df7 + 7, index=False)

    num_rows_df8 = watch_formatado.shape[0]

    inscritos_formatado.to_excel(writer, sheet_name='Resultado', startrow=num_rows_df1 + num_rows_df2 + num_rows_df3 + num_rows_df4 + num_rows_df5 + num_rows_df6 + num_rows_df7 + num_rows_df8 + 8, index=False)
    
    
    data_desvio_formatado.to_excel(writer, sheet_name='Desvio', index = False)

    num_rows_desv_df1 = data_desvio_formatado.shape[0]

    conteudo_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_desv_df1 + 1, index=False)

    num_rows_desv_df2 = conteudo_desvio_formatado.shape[0]

    receita_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_desv_df1 + num_rows_desv_df2 + 2, index=False)

    num_rows_desv_df3 = receita_desvio_formatado.shape[0]

    shorts_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + 3, index=False)

    num_rows_desv_df4 = shorts_desvio_formatado.shape[0]

    origem_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + 4, index=False)

    num_rows_desv_df5 = origem_desvio_formatado.shape[0]

    comu_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + 5, index=False)

    num_rows_desv_df6 = comu_desvio_formatado.shape[0]
    
    pubs_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + num_rows_desv_df6 + 6, index=False)

    num_rows_desv_df7 = pubs_desvio_formatado.shape[0]

    watch_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + num_rows_desv_df6 + num_rows_desv_df7 + 7, index=False)

    num_rows_desv_df8 = watch_desvio_formatado.shape[0]

    insc_desvio_formatado.to_excel(writer, sheet_name='Desvio', startrow=num_rows_df1 + num_rows_df2 + num_rows_desv_df3 + num_rows_desv_df4 + num_rows_desv_df5 + num_rows_desv_df6 + num_rows_desv_df7 + num_rows_desv_df8 + 8, index=False)



# TABELA PARA QUADRANTE

In [319]:
quadrante = pd.read_csv(f'dados_full/{artista}/total.csv')
quadrante = quadrante.drop(quadrante.index[0])
quadrante.set_index('Data', inplace=True)
quadrante = quadrante[['Inscritos','RPM (USD)','Visualizações','Receita estimada (USD)']].round(2)


insc_q_mean = quadrante['Inscritos'].mean()
rpm_q_mean = quadrante['RPM (USD)'].mean()
views_q_mean = quadrante['Visualizações'].mean()
reven_q_mean = quadrante['Receita estimada (USD)'].mean()

quadrante = quadrante.T
quadrante.insert(0, 'Média', [insc_q_mean, rpm_q_mean, views_q_mean, reven_q_mean])
quadrante.fillna(0,inplace=True)
quadrante.reset_index(inplace=True)

quadrante_desvio = quadrante.copy()
quadrante_desvio.iloc[:, 1:] = (quadrante_desvio.iloc[:, 1:].div(quadrante_desvio['Média'], axis=0) - 1)
quadrante_desvio.drop('Média', axis=1, inplace=True)
quadrante_desvio.reset_index(drop=True, inplace=True)
quadrante_desvio.insert(0, 'Data', quadrante['index'])
quadrante_desvio.fillna(0, inplace=True)
quadrante_desvio.iloc[:, 1:] = quadrante_desvio.iloc[:, 1:].replace('.', ',', regex=True)
quadrante_desvio.drop('index', axis=1, inplace=True)
quadrante_desvio

Data,Data,2023-11,2024-02,2023-12,2024-03,2024-04,2024-01
0,Inscritos,-0.008091,0.041262,-0.152104,0.132686,0.084142,-0.097896
1,RPM (USD),-0.142857,-0.115789,0.200000,0.082707,0.019549,-0.043609
2,Visualizações,0.061905,0.049445,0.000092,0.010673,-0.042349,-0.079766
3,Receita estimada (USD),-0.085182,-0.074391,0.202158,0.092376,-0.019031,-0.115930


In [320]:
with pd.ExcelWriter(f'dados_full/{artista}/quadrante.xlsx') as writer:
    
    quadrante.to_excel(writer, sheet_name='Resultado', index=False)

    quadrante_desvio.to_excel(writer, sheet_name='Desvio', index=False)